# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
len(df)

500

In [4]:
import numpy as np
import datetime

### Extraemos las fechas de acuerdo a diferentes formatos

* 04/20/2009; 04/20/09; 4/20/09; 4/3/09

In [9]:
# Extraemos por separado años en 2 y 4 dígitos para después formatear
p11 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
p12 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')

print(len(p11))
print(len(p12))

100
25


* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009

In [12]:
p2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')

print(len(p2))

34


* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Feb 2009; Sep 2009; Oct 2010

In [18]:
p3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')

print(len(p3))

184


* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009

In [20]:
p4 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z] )((?:\d{1,2}(?:st|nd|rd|th), )\d{4})')

print(len(p4))

0


* 6/2008; 12/2009

In [21]:
p6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')

print(len(p6))

137


* 2009; 2010

In [27]:
p71 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
p72 = df.str.extractall(r'^(\d{4})[^0-9]')

print(len(p71))
print(len(p72))

398
3


Una vez hemos obtenido las expresiones regulares para todos los casos, tenemos que juntar todas las fechas en el mismo dataset y ordenarlas, con las particularidades del enunciado de cada una.

Para ello, como tenemos un identificador único para cada registro, almacenaremos estos indicadores en las variables `p1_index`.

Después, juntaremos las tablas de manera que no se repitan.

** La función final es:**

In [47]:
import numpy as np
from datetime import datetime

In [48]:
def date_sorter():
    
    p11 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    p12 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    p1 = pd.concat([p11,p12])
    p1.reset_index(inplace=True)
    p1_index = p1['level_0']

    p2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    p2.reset_index(inplace=True)
    p2_index = p2['level_0']
    
    p3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    p3.reset_index(inplace=True)
    p3_index = p3['level_0']

    p6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    p6.reset_index(inplace=True)
    p6_index = p6['level_0']
    
    l = []
    for indice in p6_index:
        if not(indice in p1_index.values):
            l.append(indice)
    l = np.asarray(l)
    p6 = p6[p6['level_0'].isin(l)]

    p71= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    p72 = df.str.extractall(r'^(\d{4})[^0-9]')
    p7 = pd.concat([p71,p72])
    p7.reset_index(inplace=True)
    p7_index = p7['level_0']
    
    l=[]
    for indice in p7_index:
        if not((indice in p2_index.values) | (indice in p3_index.values) | (indice in p6_index.values)):
            l.append(indice)
    l = np.asarray(l)
    p7 = p7[p7['level_0'].isin(l)]
    
    s = p1.level_0.values.tolist()+p2.level_0.values.tolist()+p3.level_0.values.tolist()+p6.level_0.values.tolist()+p7.level_0.values.tolist()
    s = np.asarray(s)
    
    p1.columns=['level_0','match','month','day','year']
    p1['year']=p1['year'].apply(str)
    p1['year']=p1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
   
    p2[1] = p2[1].apply(lambda x: x.replace(',',''))
    p2['day'] = p2[1].apply(lambda x:x.split(' ')[0])
    p2['year'] = p2[1].apply(lambda x:x.split(' ')[1])
    p2.columns=['level_0','match','month','day-year','day','year']
    p2.drop('day-year',axis=1,inplace=True) 
    
    p3.columns=['level_0','match','day','month','year']
    p3['day'] = p3['day'].replace(np.nan,-99)
    p3['day'] = p3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    p3['month'] = p3.month.apply(lambda x: x[:3])
    p3['month'] = pd.to_datetime(p3.month, format='%b').dt.month
    
    p6.columns=['level_0','match','month','year']
    p6['day']=1
  
    p7.columns=['level_0','match','year']
    p7['day']=1
    p7['month']=1
   
    final = pd.concat([p1,p2,p3,p6,p7])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')

    myList = final['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(myList), key=lambda x:x[1])],np.arange(500))

    return answer


In [49]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64